In [2]:
import numpy as np
import sklearn
import tensorflow as tf
import pandas as pd

from keras.src.layers import Dense
from tensorflow import keras
from keras import layers
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dot, Input, Dropout, Multiply, Concatenate
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import regularizers


# Load The Data

In [3]:
users = pd.read_csv("data/ratings.csv")
users.head(5)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [4]:
from sklearn.preprocessing import LabelEncoder

# 1. Fit LabelEncoders on your raw IDs
u_enc = LabelEncoder()
m_enc = LabelEncoder()

users['u_idx'] = u_enc.fit_transform(users['userId'])
users['m_idx'] = m_enc.fit_transform(users['movieId'])

# 2. Recompute the number of unique users/items
num_users = users['u_idx'].nunique()
num_items = users['m_idx'].nunique()

# 3. Prepare inputs for the model (as 2D int32 arrays)
user_ids  = users['u_idx'].values.reshape(-1, 1).astype('int32')
movie_ids = users['m_idx'].values.reshape(-1, 1).astype('int32')
ratings = users['rating'].values.astype('float32')

# 4. (Optional) Inspect the first few rows
print(users[['userId','u_idx','movieId','m_idx']].head())

print(f"num_users={num_users}, num_items={num_items}")


   userId  u_idx  movieId  m_idx
0       1      0      110    108
1       1      0      147    145
2       1      0      858    843
3       1      0     1221   1195
4       1      0     1246   1218
num_users=270896, num_items=45115


# Collaborative Filtering Model

### When you think collaborative filtering, think of statements like:
- Users who liked similar items also liked...
- Items similar to this item

### 🧠 “Behavioral Similarity”
The system learns from what users did, not what items are about.

> It doesn’t care what genre the item is — it just learns from the pattern of user behavior.

### 🔍 How It Works:
- Looks at user-item interactions (ratings, likes, views)
- Learns latent similarities between users or items
- Powered by embeddings, matrix factorization, or neural models

finds patterns in behavior



In [5]:
gmf_dim      = 32 #defines dimensions of gmf
mlp_dim      = 32 # defines dimensions of mlp
mlp_layers   = [64,32] # mlp neuron layers

# Input layer takes item of vector size 1
user_input = Input(shape=(1,), name='userId')
item_input = Input(shape=(1,), name='movieId')

gmf_user_emb = Embedding(num_users, gmf_dim, embeddings_regularizer=regularizers.l2(1e-5))(user_input) #embedding for userID
gmf_item_emb = Embedding(num_items, gmf_dim, embeddings_regularizer=regularizers.l2(1e-5))(item_input) # embedding for movieID
gmf_user_vec = Flatten()(gmf_user_emb) # flatten them
gmf_user_vec = Dropout(0.4)(gmf_user_vec)
gmf_item_vec = Flatten()(gmf_item_emb) # flatten them
gmf_item_vec = Dropout(0.4)(gmf_item_vec)
gmf_vector   = Multiply()([gmf_user_vec, gmf_item_vec]) # multiply to check how much the movie aligns with the user in training

# 4. MLP branch
mlp_user_emb = Embedding(num_users, mlp_dim, embeddings_regularizer=regularizers.l2(1e-5))(user_input) # embedding for userID
mlp_item_emb = Embedding(num_items, mlp_dim, embeddings_regularizer=regularizers.l2(1e-5))(item_input) # embedding for movieID
mlp_user_vec = Flatten()(mlp_user_emb) # flatten them
mlp_user_vec = Dropout(0.4)(mlp_user_vec)
mlp_item_vec = Flatten()(mlp_item_emb) # flatten them
mlp_item_vec = Dropout(0.4)(mlp_item_vec)
mlp_vector   = Concatenate()([mlp_user_vec, mlp_item_vec]) # stacks them both together
for units in mlp_layers:
    mlp_vector = Dense(units, activation='relu')(mlp_vector) # goes through neurons, activation relu to allow complexity



In [7]:
# The dot product of the two vectors, which gives a single number that determines how similar the two vectors are.
fusion = Concatenate()([gmf_vector, mlp_vector]) #then we concatenate both mlp and gmf
print(fusion.shape)
output = Dense(1, activation='linear', name='prediction')(fusion) # and add them all together to make the prediction


model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse'
)
model.summary()

(None, 64)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ userId (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movieId             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 32)     │  8,668,672 │ userId[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 32)     │  1,443,680 │ movieId[0][0]     │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 32)        │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 32)        │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 32)     │  8,668,672 │ userId[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 32)     │  1,443,680 │ movieId[0][0]     │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 32)        │          0 │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 32)        │          0 │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 32)        │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 32)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ dropout_2[0][0],  │
│ (Concatenate)       │                   │            │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 32)        │          0 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,160 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 32)        │          0 │ dropout[0][0],    │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 64)        │          0 │ multiply[0][0],   │
│ (Concatenate)       │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prediction (Dense)  │ (None, 1)         │         65 │ concatenate_2[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 20,231,009 (77.18 MB)

 Trainable params: 20,231,009 (77.18 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# configure the callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',    # watch validation loss
    factor=0.5,            # multiply LR by this factor on plateau
    patience=2,            # wait this many epochs with no improvement
    min_lr=1e-6,           # don’t go below this LR
    verbose=1              # print messages when LR is reduced
)

# now include it in your fit call
model.fit(
    x=[user_ids, movie_ids],
    y=users['rating'].astype('float32'),
    epochs=10,
    batch_size=2048,
    validation_split=0.2,
    callbacks=[reduce_lr]
)


Epoch 1/10
    81/162652 ━━━━━━━━━━━━━━━━━━━━ 3:04:11 68ms/step - loss: 13.5913

KeyboardInterrupt: 

# Content-Based Filtering Model

When you think of content-based filtering, think of statements like:
- Because you liked horror
- Because you searched laptops

### 🧠 “Attribute Similarity”
The system uses the metadata or features of items (or users) directly.

> It recommends items with similar features to what you liked, not because other users liked them.

### 📦 How It Works:
- Uses item (or user) attributes: genres, categories, descriptions
- Builds a user profile from liked item features
- Compares feature vectors (e.g., via cosine similarity)

finds patterns in features

Collaborative filtering learns from who likes what, no matter what it is.

Content-based filtering learns from what the thing is, no matter who liked it.

In [ ]:
user_NN = Sequential([
    layers.Input(shape = num_user_features),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32)
])

item_NN = Sequential([
    layers.Input(shape= num_item_features),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32)
])

vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

output = layers.Dot(axes=1)([vu, vm])

model = Model([input_user, input_item], output)

cost_fn = keras.losses.MeanSquaredError()

# Training

# Testing